## ReMKiT1D input generator - Flowing cold ion collision integral test
This example tests whether the cold ion collision integral reproduces the expected analytical value in the low flow limit.

This test corresponds to the thirds test in Section 5.2.2. of the ReMKiT1D code paper.

In [ ]:
import numpy as np
import xarray as xr
import holoviews as hv
import matplotlib.pyplot as plt
import matplotlib as mpl
from holoviews import opts
import panel as pn

import sys
sys.path.append('../')
from RMK_support import RKWrapper ,Grid
import RMK_support.simple_containers as sc
import RMK_support.IO_support as io
import RMK_support.dashboard_support as ds
import RMK_support.common_models as cm



### Some useful constants

In [ ]:
elCharge = 1.60218e-19
elMass = 9.10938e-31
amu = 1.6605390666e-27  # atomic mass unit
ionMass = 2.014*amu  # deuterium mass
epsilon0 = 8.854188e-12  # vacuum permittivity
heavySpeciesMass = 2.014  # in amus


### Wrapper initialization

In [ ]:
rk = RKWrapper()


### Global parameters for IO files

In [ ]:
rk.jsonFilepath = "./config.json"  # Default value
hdf5Filepath = "./RMKOutput/RMK_CI_test/"
rk.setHDF5Path(hdf5Filepath)


### Setting options for external libraries used by ReMKiT1D

#### MPI


In [ ]:
numProcsX = 4  # Number of processes in x direction
numProcsH = 1  # Number of processes in harmonic
numProcs = numProcsX * numProcsH
haloWidth = 1  # Halo width in cells

rk.setMPIData(numProcsX, numProcsH, haloWidth)


### Normalization setup

In [ ]:
rk.setNormDensity(1.0e19)
rk.setNormTemperature(100.0)
rk.setNormRefZ(1.0)


### Grid setup

In [ ]:
xGrid = 150*np.ones(64)
dv0 = 0.0307
cv = 1.025
vGrid = [dv0]
for i in range(1,120):
    vGrid.append(vGrid[i-1]*cv)
lMax = 1
gridObj = Grid(xGrid, np.array(vGrid), lMax, interpretXGridAsWidths=True, interpretVGridAsWidths=True, isPeriodic=True)
L = sum(xGrid)

In [ ]:
# Add the grid to the wrapper
rk.grid = gridObj


### Set default species and temperature derivations

In [ ]:
rk.setStandardTextbookOptions([0])

rk.addSpecies("e", 0)
rk.addSpecies("D+", -1, atomicA=2.014, charge=1.0)


### Variables

In [ ]:
n = np.ones(gridObj.numX())
u = 0.0001*np.ones(gridObj.numX()) # Slow flow in order to reproduce the analytical solution.

T = np.ones(gridObj.numX())
W = 3*n*T/2
f = np.zeros([gridObj.numX(),gridObj.numH(),gridObj.numV()])
for i in range(gridObj.numX()):
    f[i,gridObj.getH(0)-1,:] = (np.pi*T[i])**(-1.5) * n[i]* np.exp(-gridObj.vGrid**2/T[i])

# Rescale distribution function to ensure that the numerical density moment agrees with the initial values
numerical_dens = gridObj.velocityMoment(f,0,1)
for i in range(gridObj.numX()):
    f[i,gridObj.getH(0)-1,:] = n[i] *f[i,gridObj.getH(0)-1,:]/numerical_dens[i]
    
rk.addVarAndDual("f",f,isDistribution=True,isCommunicated=True)
rk.addVar("W",W,units='$10eV$',isDerived=True,derivationRule=sc.derivationRule("energyMoment",["f"]))
rk.addVarAndDual("n",n,units='$10^{19} m^{-3}$',isDerived=True,derivationRule=sc.derivationRule("densityMoment",["f"]))
rk.addVarAndDual("ni",n)
rk.addVarAndDual("Gi",n*u,primaryOnDualGrid=True)
rk.addVar("ui",u,isDerived=True,derivationRule=sc.derivationRule("flowSpeedFromFlux",["Gi_dual","ni_dual"]))
rk.addVarAndDual("G",isDerived=True,primaryOnDualGrid=True,derivationRule=sc.derivationRule("fluxMoment",["f"]))
rk.addVarAndDual("u",isDerived=True,primaryOnDualGrid=True,derivationRule=sc.derivationRule("flowSpeedFromFlux",["G_dual","n_dual"]),isCommunicated=True)
rk.addVarAndDual("T",T,isDerived=True,derivationRule=sc.derivationRule("tempFromEnergye",["W","n","G"]),isCommunicated=True)
rk.addVar("time",isScalar=True,isDerived=True)


## Adding the flowing cold ion electron-ion model

This model is implemented in common_models.py and only used here.

In [ ]:
cm.addFlowingIonEIColl(modelTag="e-i_odd",
                          distFunName="f",
                          ionDensVar="ni",
                          ionFlowSpeedVar="ui",
                          electronDensVar="n_dual",
                          electronTempVar="T_dual",
                          ionSpeciesName="D+",
                          evolvedHarmonics=list(range(2, gridObj.numH()+1, 2)),
                          wrapper=rk,
                          dualDistFun="f_dual",
                          ionFluxVar="Gi_dual")

### Integrator and timestep options

Simple single step backwards Euler integration

In [ ]:
# Everything default except for more lenient absolute convergence tolerance
integrator = sc.picardBDEIntegrator(absTol=10.0, convergenceVars=["f","Gi_dual"],nonlinTol=1e-14)

rk.addIntegrator("BE", integrator)


Set initial timestep length and numbers of allowed implicit and general groups

In [ ]:
initialTimestep = 0.1

rk.setIntegratorGlobalData(1,1, initialTimestep)


Single integration step

In [ ]:
bdeStep = sc.IntegrationStep("BE")

for tag in rk.modelTags():
    bdeStep.addModel(tag)

rk.addIntegrationStep("BE1", bdeStep.dict())


#### Timeloop options

In [ ]:
rk.setFixedNumTimesteps(1000)
rk.setFixedStepOutput(100)


### Create config file

In [ ]:
rk.writeConfigFile()


### Data analysis


In [ ]:
numFiles = 10

#### Loading data

Set loadpath to ReMKiT1D directory

In [ ]:
loadpath = hdf5Filepath
loadFilenames = [loadpath+f'ReMKiT1DVarOutput_{i}.h5' for i in range(numFiles+1)]


In [ ]:
loadedData = io.loadFromHDF5(rk.varCont, filepaths=loadFilenames)
loadedData


In [ ]:
loadedData.coords['x'].attrs['units'] = "$x_0$"
loadedData.coords['v'].attrs['units'] = "$v_{th}$"
loadedData.coords['time'].attrs['standard_name'] = 't'
loadedData.coords['time'].attrs['units'] = "$t_0$"


In [ ]:
hv.extension('matplotlib')
%matplotlib inline
plt.rcParams['figure.dpi'] = 150
hv.output(size=80, dpi=150)


### Relative velocity error

In [ ]:
abs(loadedData["u"].data[-1,0]-loadedData["ui"].data[-1,0])/loadedData["ui"].data[-1,0]

### Compare with analytic solution

In [ ]:
f_analytic = np.zeros([gridObj.numX(),gridObj.numV()])
final_ue=loadedData["u"].data[-1,:]
for i in range(gridObj.numX()):
    f_analytic[i,:] = 2*final_ue[i]*gridObj.vGrid *(np.pi*T[i])**(-1.5) * n[i]* np.exp(-gridObj.vGrid**2/T[i])/T[i]

In [ ]:
dataName = 'f'
curve = hv.Curve((gridObj.vGrid,f_analytic[0,:]),label='Analytical formula') *hv.Scatter((gridObj.vGrid,loadedData["f"].data[-1,0,1,:]),label='Numerical result').opts(s=6.0,color='r',marker='x')
curve.opts(xlabel='v [$v_{th}$]',ylabel='f$_1$ [normalized units]')

### Relative total momentum error

In [ ]:
initMom = loadedData["G_dual"][0,0]*elMass/ionMass +loadedData["Gi_dual"][0,0]   # initial momentum in electron units
endMom = loadedData["G_dual"][:,0]*elMass/ionMass +loadedData["Gi_dual"][:,0]  # final momentum in electron units

momErr=abs(endMom-initMom)/initMom

In [ ]:
momErr

In [ ]:
hv.output(fig='pdf')
hv.save(curve.opts(xlabel='v [$v_{th}$]',ylabel='f$_1$ [normalized units]'), 'e-if1Relaxation.pdf', dpi=144)